# NARMAX Notes and Codes
### Liam M. Kilcommons

## Simple Orthogonal Least Squares

### Model components
Assume we have N observations of some arbitrary number of variables $x$
$$\vec{x}_k=[x_{1,k},x_{2,k},...]^{T}$$ 
$$k \in [1,N]$$

Suppose further we wish to create an M term/parameter nonlinear model with terms $p$, where each $p$ is some function of the observed variables $x$:
$$\vec{p}_{k} = [p_{1,k},p_{2,k},...] $$
$$p_{i,k} = f_{i}([x_{1,k},x_{2,k},...]^{T})$$
$$i \in [1,M]$$

### Model Equation
The (linear-in-the-parameters) model can be represented with the following matrix equation:
$$Y_{[N,1]}=P_{[N,M]}\theta_{[M,1]} + e_{[N,1]}$$
Where $e$ is an error term

### Orthogonal Auxillary Model
We will create an auxillary model represented by the following matrix equation:
$$Y_{[N,1]}=W_{[N,M]}g_{[M,1]} + e_{[N,1]}$$
Where the columns of W are orthogonal:
$$
    \langle W[:,i],W[:,j] \rangle = 
        \begin{cases} 
            d_{i} & i = j \\
            0 & i \ne j
        \end{cases}
$$
($\langle .,. \rangle$ is the inner (dot) product)  

Note that this also implies that:
$$W^{T}W = \Lambda = diag([d_1,d_2,...d_M])$$

If the regression matrix P from the original model is full rank in columns (its columns span an M dimensional vector space)  
Then P can be QR decomposed into a matrix with orthogonal columns $W$ and an upper triangular matrix $A$:
$$P_{[N,M]} = W_{[N,M]}A_{[M,M]}$$

It can be shown that the following relationship between the original regression equation and the auxillary one holds:
$$ Y = Wg + e = (PA^{-1})(A\theta) + e $$

Then, due to orthogonality, performing the auxillary regression to get coefficient vector $g$ becomes a simple matrix multipliction:  

$$ g_{[M,1]} = \Lambda_{[M,M]}^{-1}W^{T}_{[M,N]}Y_{[N,1]} $$

Then the original model coefficient vectors $\theta$ can be calculated by performing Gram-Schmidt on the following:
$$ A_{[M,M]}\theta_{[M,1]} = g_{[M,1]} $$


# Foreward Regression Orthongonal Least Squares (FROLS)

As in simple OLS above, let the N values of M model terms $p$ (functions of N observations of arbitrary number of model variables $x$) be:
   $$p_{i}(k), k \in [1,N], i \in [1,M]$$
   
In matrix notation:
   $$P_{[N,M]}[k,i] = p_{i}(k)$$

Also, let the overall regression be: the column vector of outputs you want to predict:
    $$Y_{[N,1]} = P_{[N,M]}\theta_{[1,M]} + e_{[1,N]}$$

Then, let the columns of P form a set D:
    $$D = \{P[:,1],P[:,2],...,P[:,M]\}$$
    
## Step 1: Select First Significant Model Term
Define:
    $$ D^{1} = D $$
    $$ Q = P $$
Find regression coefficients and error reducing ratio:
    $$ g^{(1)}_{[1,M]}[1,m] = \frac{ \langle Y[:,1],Q^{(1)}[:,m] \rangle }{ \langle Q^{(1)}[:,m],Q^{(1)}[:,m] \rangle }$$
    $$ ERR^{(1)}_{[1,M]}[1,m] = g[1,m]^{2} \frac{ \langle Q^{(1)}[:,m],Q^{(1)}[:,m] \rangle }{ \langle Y^{(1)}[:,1],Y^{(1)}[:,1] \rangle}$$
    $$ m \in [1,M] $$
    
Find the index $l \in [1,M]$ which maximizes ERR:
$$ l_{1} = argmax(ERR^{(1)}[1,:]) $$

## Step 2,3,... : Select More Significant Model Terms
Define, for s-th step:
    $$ D^{(s-1)}=\{P[:,l_{1}],P[:,l_{2}],...,P[:,l_{s-1}]\} $$  
    $$ D = D - D^{(s-1)}$$
Define $Q$ as a matrix of orthogonal columns spanning the same space as the already selected columns of $P$ (QR factorization):
    $$P^{(s-1)} = QR $$
    $$P^{(s-1)}=[P[:,l_{1}],P[:,l_{2}],...,P[:,l_{s-1}]]$$
Remove the information in $P^{(s-1)}$, via $Q$ from the remaining (unselected) columns of P:
    $$ Q^{(s)}_{[N,M_{s}]}[:,m] = P[:,m] - \sum_{r=1}^{M0} \frac{ \langle P[:,m],Q[:,r] \rangle }{ \langle Q[:,r], Q[:,r]\rangle } Q[:,r] $$  

Find the s-th-iteration-specific regression coefficients and Error Reducing Ratio:
    $$ M_{s} = M-s $$
    $$ g^{(s)}_{[1,M_{s}]}[1,m] = \frac{ \langle Y[:,1],Q^{(s)}[:,m] \rangle }{ \langle Q^{(s)}[:,m],Q^{(s)}[:,m] \rangle }$$
    $$ ERR^{(s)}_{[1,M_{s}]}[1,m] = g^{(s)}[1,m]^{2} \frac{ \langle Q^{(s)}[:,m],Q^{(s)}[:,m] \rangle }{ \langle Y[:,1],Y[:,1] \rangle}$$
    $$ m \in [1,M] \text{ excluding } [l_1,l_2,...,l_s] $$
    
## Penultimate Step, Put Together Final Auxillary Model

Define final orthogonal auxillary term matrix (assume M0 terms were found):
    $$ Q_{[N,M0]} = [Q^{1}[:,l_1],Q^{2}[:,l_2],...,Q^{M0}[:,l_{M0}]] $$
Define M0 final auxillary regression coefficients:
    $$ g_{[1,M0]} = [g^{1}[:,l_1],g^{2}[:,l_2],...,g^{M0}[:,l_{M0}]] $$
Define final triangular matrix (i.e. the R of QR):
    $$ A_{[M0,M0]}[r,s] =  \frac{ \langle Q[:,r],P[:,l_s] \rangle }{ \langle Q[:,r], Q[:,r]\rangle } r \in [1,s-1]$$
    $$ A_{[M0,M0]}[s,s] = 1 $$
Define final ERR:
    $$ ERR_{[1,M0]}[1,s] = ERR^{(s)}[1,l_s] $$ 
    
## Final Step: Transform Final Auxillary Model To Final Model

Define the final model as:
$$ Y_{[N,M0]} = P_{l,[N,M0]}\beta_{l,[M0,1]} + e_{[M0,1]} $$
$$ P_{l} = [P[:,l_1],P[:,l_2],...,P[:,l_{M0}]] $$ 
To find coefficients $\beta$ solve (scipy.linalg.solve_triangular):
$$ A\beta = g $$




In [193]:
import numpy as np
import scipy.linalg as linalg
def inner_product_ratio(x,y):
    return np.dot(x.T,y)/np.dot(y.T,y)
def proj(u,v):
    return np.dot(u.T,v)/np.dot(u.T,u)*u
def calc_g(Q,Y):
    n_params = Q.shape[1]
    g = np.full((n_params,1),np.nan)
    for i_param in range(n_params):
        g[i_param]=inner_product_ratio(Y,Q[:,i_param])
    return g
def calc_err(g,Q,Y):
    sigma = np.dot(Y.T,Y)
    n_params = Q.shape[1]
    err = np.full_like(g,np.nan)
    for i_param in range(n_params):
        err[i_param]=g[i_param]**2*np.dot(Q[:,i_param].T,Q[:,i_param])/sigma
    return err
def calc_esr(err):
    return 1-np.sum(err.flatten())
def orthogonalize(P):
    (Q,R) = np.linalg.qr(P,mode='reduced')
    return Q

def remove_B_columns_from_A(A,B):
    n_params = A.shape[1]
    n_cols = B.shape[1]
    A_new = A.copy()
    for i_param in range(n_params):
        for i_col in range(n_cols):
            A_new[:,i_param] -= proj(B[:,i_col],A[:,i_param])
    return A_new
def solve_triangular(A,B):
    """
    Solve Ax = B for x when A is upper triangular, and has unit diagonal
    """
    x = linalg.solve_triangular(A,B,unit_diagonal=True)
    return x
def frols(Y,P,pcols=None,term_thresh=1.0e-3,max_steps=1000):
    l_list = [] # List of indicies (columns of P) which have been selected for model
    g_list = [] # List of coefficients to retained from each step (i.e. for step s g^s[:,l_s])
    Q_list = [] # List of columns of step-specific orthogonal matrix Q columns to retain (i.e. Q^s[:,l_s])
    err_list = [] #List of step-specific error reducing ratios to retain (ERR^s[:,l_s])
    if pcols is None:
        pcols = {i:'x%d' % (i+1) for i in range(P.shape[1])}
    
    #Mask
    selected = np.zeros((P.shape[1],),dtype=bool)
    esr = 1.
    i_step = 0
    while esr > term_thresh and i_step<P.shape[1]:
        selected[l_list] = True
        unselected = np.logical_not(selected)
        unselected_inds = np.flatnonzero(unselected)
        print("Not selected %s" % (str(unselected_inds)))
        if i_step==0:
            Q = P.copy()
            g = calc_g(Q,Y)
            err = calc_err(g,Q,Y)
            l = np.argmax(err)
            l_list.append(l)
            g_list.append(g[l])
            Q_list.append(Q[:,l])
            err_list.append(err[l])
        else:
            print("Shape of P = %s" % (str(P[:,unselected].shape)))
            Q = orthogonalize(P[:,selected])
            thisQ = remove_B_columns_from_A(P[:,unselected],Q)
            g = calc_g(thisQ,Y)
            err = calc_err(g,thisQ,Y)
            thisl = np.argmax(err)
            l = unselected_inds[thisl]
            l_list.append(l)
            g_list.append(g[thisl])
            Q_list.append(thisQ[:,thisl])
            err_list.append(err[thisl])
        esr = calc_esr(np.array(err_list))
        print("Step %d: Selected term at index %d (%s), ERR=%f, ESR=%f" % (i_step,l_list[-1],pcols[l_list[-1]],err_list[-1],esr))
        i_step+=1
    #Finally put together the final coefficients
    model_str = []
    Q_f = np.column_stack(Q_list)
    g_f = np.array(g_list).reshape(-1,1)
    err_f = np.array(err_list).reshape(-1,1)
    n_identified = Q_f.shape[1]
    A = np.eye(n_identified)
    Pl = P[:,l_list]
    #Fill in upper triangluar part of A
    for s in range(n_identified):
        for r in range(s):
            if r==s:
                A[r,s]=1.
            else:
                A[r,s] = np.dot(Q_f[:,r].T,Pl[:,s])/np.dot(Q_f[:,r].T,Q_f[:,r])
    coef = solve_triangular(A,g_f)
    pred_Y = np.dot(Pl,coef)
    modelstr = '+'.join(['%.9f*%s' % (coef[k],pcols[l_list[k]]) for k in range(n_identified)])
    print("Using model: %s\n RMSE is %f" % (modelstr,np.sqrt(np.mean((Y-pred_Y)**2))) )
    full_coef = np.zeros((P.shape[1],1))
    full_err = np.zeros((P.shape[1],1))
    full_coef[l_list]=coef
    full_err[l_list]=err_f
    return full_coef,full_err,pred_Y

In [132]:
#Pg. 63 of Billings, NARMAX Methods
#Example data for NARMAX test
X = np.array([[9,-5,5,-1.53],
              [1,-1,8,-0.39],
              [2,-5,6,-3.26],
              [8,-2,0,0.36],
              [0,0,9,0.13]])
Y = np.array([[9.08],
              [7.87],
              [3.01],
              [5.98],
              [9.05]])
P = X.copy()
pcols = {0:'x1',1:'x2',2:'x3',3:'x4',4:'x5'}
frols(Y,P,pcols=pcols)    

Not selected [0 1 2 3]
Step 0: Selected term at index 2 (x3), ERR=0.773707, ESR=0.226293
Not selected [0 1 3]
Shape of P = (5, 3)
Step 1: Selected term at index 0 (x1), ERR=0.172684, ESR=0.053609
Not selected [1 3]
Shape of P = (5, 2)
Step 2: Selected term at index 1 (x2), ERR=0.053523, ESR=0.000086
[[ 1.          0.31553398 -0.30582524]
 [ 0.          1.         -0.40251172]
 [ 0.          0.          1.        ]]
Using model: 0.9967*x3+1.0005*x1+0.9917*x2
 RMSE is 0.068330


(array([[ 1.00046032],
        [ 0.99172293],
        [ 0.99669235],
        [ 0.        ]]), array([[ 0.17268374],
        [ 0.05352266],
        [ 0.77370749],
        [ 0.        ]]))